In [1]:
import math

import pandas as pd
import numpy as np
import json
import ast
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

In [2]:
# 今回使う説明変数を決める
x_col = ["genres","original_language","production_companies","release_date","runtime","vote_average"]
df = pd.read_csv("movie/tmdb_5000_movies.csv")
df_x = df[x_col]

In [3]:
df_x.head()

,genres,original_language,production_companies,release_date,runtime,vote_average
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,162.0,7.2
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",en,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,169.0,6.9
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,148.0,6.3
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",en,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,165.0,7.6
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,132.0,6.1


In [4]:
#ではここから全てをfloat型に変えていく
genres = df_x["genres"]
# とりあえず種類が欲しいので種類を確認
# まずは辞書を作成
movie_genre_dict = dict()
movie_genre_list = list()

for movie_genre in genres:
    if movie_genre == "[]":
        movie_genre_list.append("")
    else:
        per_genre_list = ast.literal_eval(movie_genre[1:-1])
        genre_list = []
        for per_genre in per_genre_list:
            if type(per_genre) == dict:
                movie_id = per_genre["id"]
                movie_name = per_genre["name"]
                movie_genre_dict[movie_id] = movie_name
                genre_list.append(movie_name)
            else:
                movie_id = per_genre_list["id"]
                movie_name = per_genre_list["name"]
                movie_genre_dict[movie_id] = movie_name
                genre_list.append(movie_name)
                break
        movie_genre_list.append(genre_list)
        genre_list = []

In [5]:
print(len(genres))
print(len(movie_genre_list))

4803
4803


In [6]:
# 20種類のジャンルがあることがわかった
# ここからワンホットエンコーディングを行う

In [7]:
genres_list = ["Action","Adventure","Fantasy","Science Fiction","Crime","Drama","Thriller","Animation","Family","Western","Comedy","Romance","Horror","Mystery","History","War","Music","Documentary","Foreign","TV Movie"]

In [8]:
a = [0]*4803*20
df_genre = pd.DataFrame(np.array(a).reshape(4803, 20),
                  columns=genres_list)
df_genre.head()

,Action,Adventure,Fantasy,Science Fiction,Crime,Drama,Thriller,Animation,Family,Western,Comedy,Romance,Horror,Mystery,History,War,Music,Documentary,Foreign,TV Movie
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
movie_genre_list

[['Action', 'Adventure', 'Fantasy', 'Science Fiction'],
 ['Adventure', 'Fantasy', 'Action'],
 ['Action', 'Adventure', 'Crime'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Fantasy', 'Action', 'Adventure'],
 ['Animation', 'Family'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Adventure', 'Fantasy', 'Family'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Adventure', 'Fantasy', 'Action', 'Science Fiction'],
 ['Adventure', 'Action', 'Thriller', 'Crime'],
 ['Adventure', 'Fantasy', 'Action'],
 ['Action', 'Adventure', 'Western'],
 ['Action', 'Adventure', 'Fantasy', 'Science Fiction'],
 ['Adventure', 'Family', 'Fantasy'],
 ['Science Fiction', 'Action', 'Adventure'],
 ['Adventure', 'Action', 'Fantasy'],
 ['Action', 'Comedy', 'Science Fiction'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure'],
 ['Adventure', 'Fantasy'],
 ['Adventure', 'Fantasy'],
 ['Adventure', 'Drama', 'Action'],
 ['Drama', 'Romance

In [10]:
# df.at[0,"Action"]とすると任意の値を取得できる
def put_genre(df,movie_genre_list):
    for i in range(len(df)):
        for genre in movie_genre_list[i]:
            df.at[i,genre] = 1.0
put_genre(df_genre,movie_genre_list)

In [11]:
df_genre.tail()

,Action,Adventure,Fantasy,Science Fiction,Crime,Drama,Thriller,Animation,Family,Western,Comedy,Romance,Horror,Mystery,History,War,Music,Documentary,Foreign,TV Movie
4798,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4799,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
4800,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1
4801,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4802,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [12]:
language = df["original_language"]
language.value_counts()

en    4505
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ru      11
ko      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [13]:
# 映画が造られた国をラベルエンコーディング
le = LabelEncoder()

In [14]:
df_x["original_language"] = le.fit_transform(df_x["original_language"])

/var/folders/38/75h5lv4d3_5bm79gp3zs7l4r0000gn/T/ipykernel_62841/963953020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x["original_language"] = le.fit_transform(df_x["original_language"])


In [15]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   original_language     4803 non-null   int64  
 2   production_companies  4803 non-null   object 
 3   release_date          4802 non-null   object 
 4   runtime               4801 non-null   float64
 5   vote_average          4803 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 225.3+ KB


In [16]:
df_x.drop(columns="genres")
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   original_language     4803 non-null   int64  
 2   production_companies  4803 non-null   object 
 3   release_date          4802 non-null   object 
 4   runtime               4801 non-null   float64
 5   vote_average          4803 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 225.3+ KB


In [17]:
# 次はリリース日を数値型へと変換していく
# また今回は月と年数に分ける
df["release_date"].value_counts()

2006-01-01    10
2002-01-01     8
2004-09-03     7
1999-10-22     7
2013-07-18     7
              ..
2002-12-30     1
2002-08-20     1
1987-11-05     1
2004-11-11     1
2012-05-03     1
Name: release_date, Length: 3280, dtype: int64

In [18]:
# 変換するための辞書を作成していく
release_year = dict()
release_month = dict()
for key in df["release_date"]:
    if type(key) == str:
        year,month,day = key.split("-")
        release_year[key] = float(year)
        release_month[key] = float(month)

In [19]:
df_x["release_year"] = df_x["release_date"].replace(release_year).astype(float)
df_x["release_month"] = df_x["release_date"].replace(release_month).astype(float)
df_x.info()

/var/folders/38/75h5lv4d3_5bm79gp3zs7l4r0000gn/T/ipykernel_62841/2932335168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x["release_year"] = df_x["release_date"].replace(release_year).astype(float)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   original_language     4803 non-null   int64  
 2   production_companies  4803 non-null   object 
 3   release_date          4802 non-null   object 
 4   runtime               4801 non-null   float64
 5   vote_average          4803 non-null   float64
 6   release_year          4802 non-null   float64
 7   release_month         4802 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 300.3+ KB


/var/folders/38/75h5lv4d3_5bm79gp3zs7l4r0000gn/T/ipykernel_62841/2932335168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x["release_month"] = df_x["release_date"].replace(release_month).astype(float)


In [20]:
# 評価値をラベルエンコーディング
# 具体的には10段階で評価する
# 小数点以下は切り捨て
reviews = df_x["vote_average"]
review_dict = dict()
for review in reviews:
    if review == 0:
        review_dict[review] = np.nan
    else:
        review_point = math.floor(review)
        review_dict[review] = review_point
review_dict

{7.2: 7,
 6.9: 6,
 6.3: 6,
 7.6: 7,
 6.1: 6,
 5.9: 5,
 7.4: 7,
 7.3: 7,
 5.7: 5,
 5.4: 5,
 7.0: 7,
 6.5: 6,
 6.4: 6,
 6.2: 6,
 7.1: 7,
 5.8: 5,
 6.6: 6,
 7.5: 7,
 5.5: 5,
 6.7: 6,
 6.8: 6,
 6.0: 6,
 5.1: 5,
 7.8: 7,
 5.6: 5,
 5.2: 5,
 8.2: 8,
 7.7: 7,
 5.3: 5,
 8.0: 8,
 4.8: 4,
 4.9: 4,
 7.9: 7,
 8.1: 8,
 4.7: 4,
 5.0: 5,
 4.2: 4,
 4.4: 4,
 4.1: 4,
 3.7: 3,
 3.6: 3,
 3.0: 3,
 3.9: 3,
 4.3: 4,
 4.5: 4,
 3.4: 3,
 4.6: 4,
 8.3: 8,
 3.5: 3,
 4.0: 4,
 2.3: 2,
 3.2: 3,
 0.0: nan,
 3.8: 3,
 2.9: 2,
 8.5: 8,
 1.9: 1,
 3.1: 3,
 3.3: 3,
 2.2: 2,
 0.5: 0,
 9.3: 9,
 8.4: 8,
 2.7: 2,
 10.0: 10,
 1.0: 1,
 2.0: 2,
 2.8: 2,
 9.5: 9,
 2.6: 2,
 2.4: 2}

In [21]:
# 上記の数値を埋め込む
df_x["vote_average"] = df_x["vote_average"].replace(review_dict).astype(float)
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   original_language     4803 non-null   int64  
 2   production_companies  4803 non-null   object 
 3   release_date          4802 non-null   object 
 4   runtime               4801 non-null   float64
 5   vote_average          4740 non-null   float64
 6   release_year          4802 non-null   float64
 7   release_month         4802 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 300.3+ KB


/var/folders/38/75h5lv4d3_5bm79gp3zs7l4r0000gn/T/ipykernel_62841/2295790603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x["vote_average"] = df_x["vote_average"].replace(review_dict).astype(float)


In [22]:
# 制作会社をラベルエンコーディング
production_companies = df_x["production_companies"]

In [23]:
# まずは空のものがいくつあるのかを確認
empty_list = list()
companies_dict = dict()
companies_list = list()
for i,companies in enumerate(production_companies):
    if companies == "[]":
        empty_list.append(i)
        companies_list.append("")
    else:
        companies = ast.literal_eval(companies[1:-1])
        append_list = []
        for j,company in enumerate(companies):
            if type(company) == dict:
                name = company["name"]
                if name in companies_dict:
                    companies_dict[name] += 1
                else:
                    companies_dict[name] = 1
                append_list.append(name)
            else:
                name = companies["name"]
                if name in companies_dict:
                    companies_dict[name] += 1
                else:
                    companies_dict[name] = 1
                append_list.append(name)
    companies_list.append(append_list)
    append_list = []
companies_list

[['Ingenious Film Partners',
  'Twentieth Century Fox Film Corporation',
  'Dune Entertainment',
  'Lightstorm Entertainment'],
 ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Second Mate Productions'],
 ['Columbia Pictures', 'Danjaq', 'B24'],
 ['Legendary Pictures', 'Warner Bros.', 'DC Entertainment', 'Syncopy'],
 ['Walt Disney Pictures', 'Walt Disney Pictures'],
 ['Columbia Pictures', 'Laura Ziskin Productions', 'Marvel Enterprises'],
 ['Walt Disney Pictures', 'Walt Disney Animation Studios'],
 ['Marvel Studios', 'Prime Focus', 'Revolution Sun Studios'],
 ['Warner Bros.', 'Heyday Films'],
 ['DC Comics',
  'Atlas Entertainment',
  'Warner Bros.',
  'DC Entertainment',
  'Cruel & Unusual Films',
  'RatPac-Dune Entertainment'],
 ['DC Comics',
  'Legendary Pictures',
  'Warner Bros.',
  'Bad Hat Harry Productions'],
 ['Eon Productions', 'Eon Productions'],
 ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Second Mate Productions'],
 ['Walt Disney Pictures',
  'Jerry Bru

In [24]:
production_companies[0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [25]:
# 流石に会社が多すぎるため,1つのものは消していきたい
new_companies_dict = dict()
new_companies_list = list()
for key,value in companies_dict.items():
    if value <= 50:
        continue
    else:
        new_companies_dict[key] = value
        new_companies_list.append(key)
len(new_companies_dict)

25

In [26]:
new_companies_dict

{'Twentieth Century Fox Film Corporation': 253,
 'Dune Entertainment': 59,
 'Walt Disney Pictures': 141,
 'Columbia Pictures': 238,
 'Warner Bros.': 346,
 'Paramount Pictures': 343,
 'New Line Cinema': 203,
 'Metro-Goldwyn-Mayer (MGM)': 154,
 'Universal Pictures': 356,
 'Relativity Media': 105,
 'DreamWorks SKG': 85,
 'Village Roadshow Pictures': 82,
 'Lionsgate': 61,
 'Fox 2000 Pictures': 65,
 'Touchstone Pictures': 144,
 'Columbia Pictures Corporation': 112,
 'Summit Entertainment': 56,
 'Regency Enterprises': 69,
 'TriStar Pictures': 61,
 'Canal+': 75,
 'Miramax Films': 110,
 'Working Title Films': 55,
 'United Artists': 87,
 'Dimension Films': 71,
 'Fox Searchlight Pictures': 73}

In [27]:
# では配給会社をもとにデータフレームを構築する
a = [0]*4803*25
df_companies = pd.DataFrame(np.array(a).reshape(4803, 25),
                  columns=new_companies_list)
df_companies.head()

,Twentieth Century Fox Film Corporation,Dune Entertainment,Walt Disney Pictures,Columbia Pictures,Warner Bros.,Paramount Pictures,New Line Cinema,Metro-Goldwyn-Mayer (MGM),Universal Pictures,Relativity Media,...,Columbia Pictures Corporation,Summit Entertainment,Regency Enterprises,TriStar Pictures,Canal+,Miramax Films,Working Title Films,United Artists,Dimension Films,Fox Searchlight Pictures
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# 作成したデータフレームに値を格納していく
def put_companies(df,companies_list):
    for i in range(len(df)):
        for company in companies_list[i]:
            if company in df.columns.values:
                df.at[i,company] = 1.0
put_companies(df_companies,companies_list)
df_companies

,Twentieth Century Fox Film Corporation,Dune Entertainment,Walt Disney Pictures,Columbia Pictures,Warner Bros.,Paramount Pictures,New Line Cinema,Metro-Goldwyn-Mayer (MGM),Universal Pictures,Relativity Media,...,Columbia Pictures Corporation,Summit Entertainment,Regency Enterprises,TriStar Pictures,Canal+,Miramax Films,Working Title Films,United Artists,Dimension Films,Fox Searchlight Pictures
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   original_language     4803 non-null   int64  
 2   production_companies  4803 non-null   object 
 3   release_date          4802 non-null   object 
 4   runtime               4801 non-null   float64
 5   vote_average          4740 non-null   float64
 6   release_year          4802 non-null   float64
 7   release_month         4802 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 300.3+ KB


In [30]:
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Action           4803 non-null   int64
 1   Adventure        4803 non-null   int64
 2   Fantasy          4803 non-null   int64
 3   Science Fiction  4803 non-null   int64
 4   Crime            4803 non-null   int64
 5   Drama            4803 non-null   int64
 6   Thriller         4803 non-null   int64
 7   Animation        4803 non-null   int64
 8   Family           4803 non-null   int64
 9   Western          4803 non-null   int64
 10  Comedy           4803 non-null   int64
 11  Romance          4803 non-null   int64
 12  Horror           4803 non-null   int64
 13  Mystery          4803 non-null   int64
 14  History          4803 non-null   int64
 15  War              4803 non-null   int64
 16  Music            4803 non-null   int64
 17  Documentary      4803 non-null   int64
 18  Foreign 

In [31]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   Twentieth Century Fox Film Corporation  4803 non-null   int64
 1   Dune Entertainment                      4803 non-null   int64
 2   Walt Disney Pictures                    4803 non-null   int64
 3   Columbia Pictures                       4803 non-null   int64
 4   Warner Bros.                            4803 non-null   int64
 5   Paramount Pictures                      4803 non-null   int64
 6   New Line Cinema                         4803 non-null   int64
 7   Metro-Goldwyn-Mayer (MGM)               4803 non-null   int64
 8   Universal Pictures                      4803 non-null   int64
 9   Relativity Media                        4803 non-null   int64
 10  DreamWorks SKG                          4803 non-null   int64
 11  Village Roadshow 

In [32]:
df_x = df_x.drop(columns=["genres","production_companies","release_date",])

In [33]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_language  4803 non-null   int64  
 1   runtime            4801 non-null   float64
 2   vote_average       4740 non-null   float64
 3   release_year       4802 non-null   float64
 4   release_month      4802 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 187.7 KB


In [34]:
df_x = pd.concat([df_x,df_companies,df_genre],axis=1)

In [35]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 50 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   original_language                       4803 non-null   int64  
 1   runtime                                 4801 non-null   float64
 2   vote_average                            4740 non-null   float64
 3   release_year                            4802 non-null   float64
 4   release_month                           4802 non-null   float64
 5   Twentieth Century Fox Film Corporation  4803 non-null   int64  
 6   Dune Entertainment                      4803 non-null   int64  
 7   Walt Disney Pictures                    4803 non-null   int64  
 8   Columbia Pictures                       4803 non-null   int64  
 9   Warner Bros.                            4803 non-null   int64  
 10  Paramount Pictures                      4803 non-null   int6

In [36]:
# まずは欠損値があるものを削除していく(今回はもう欠損値処理せずに消す)
df_x = df_x.dropna(axis=0,how="any")
df_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4739 entries, 0 to 4802
Data columns (total 50 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   original_language                       4739 non-null   int64  
 1   runtime                                 4739 non-null   float64
 2   vote_average                            4739 non-null   float64
 3   release_year                            4739 non-null   float64
 4   release_month                           4739 non-null   float64
 5   Twentieth Century Fox Film Corporation  4739 non-null   int64  
 6   Dune Entertainment                      4739 non-null   int64  
 7   Walt Disney Pictures                    4739 non-null   int64  
 8   Columbia Pictures                       4739 non-null   int64  
 9   Warner Bros.                            4739 non-null   int64  
 10  Paramount Pictures                      4739 non-null   int6

In [37]:
df_x["vote_average"].nunique()

11

In [38]:
# とりあえずテストデータと検証データに分ける

# 説明変数,目的変数
X = df_x.drop("vote_average",axis=1).values # 説明変数(target以外の特徴量)
y = df_x["vote_average"].values # 目的変数(target)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=2)


lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass', # 目的 : 多クラス分類
        'num_class': 11, # クラス数 : 11
        'metric': {'multi_error'}, # 評価指標 : 誤り率(= 1-正答率)
        # 他には'multi_logloss'など
}

In [ ]:
# モデルの学習
model = lgb.train(params,
train_set=lgb_train, # トレーニングデータの指定
valid_sets=lgb_eval, # 検証データの指定
)

# テストデータの予測 ((各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す))
y_pred_prob = model.predict(X_test)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに

In [ ]:
# 真値と予測値の表示
df_pred = pd.DataFrame({'target':y_test,'target_pred':y_pred})
display(df_pred)

# 真値と予測確率の表示
df_pred_prob = pd.DataFrame({'y':y_test, 'target0_prob':y_pred_prob[:,0], 'target1_prob':y_pred_prob[:,1], 'target2_prob':y_pred_prob[:,2]})
display(df_pred_prob)

In [ ]:
#予測値の確認
# 真値と予測値の表示
from sklearn.metrics import accuracy_score
# モデル評価
# acc : 正答率
acc = accuracy_score(y_test,y_pred)
print('Acc :', acc)